In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import chi2
import pickle
import random
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch.nn import functional as F 

In [2]:
torch.cuda.is_available()

True

In [3]:
lr = 0.001
epochs = 50
batch_size = 20000

#IMPORTING DATA
#dataset variable should be a tuple of form (data, targets), each 1e5 long
with open('./datasets/list_orbital_dataset.pickle', 'rb') as data:
    dataset = pickle.load(data)

data, targets = dataset

#GET THE DEVICE (PREFERABLY A GPU)
def get_device():
    if torch.cuda.is_available():
        device = 'cuda:0'
    else:
        device = 'cpu'
    return device

#CREATING A CUSTOM TRAINLOADER
class DatasetClass(Dataset):
  def __init__(self,x,y):
    self.x = torch.tensor(x,dtype=torch.float32)
    self.y = torch.tensor(y,dtype=torch.float32)
    self.length = self.x.shape[0]
 
  def __getitem__(self,idx):
    return self.x[idx],self.y[idx]
  def __len__(self):
    return self.length

#CUSTOM TRAIN TEST SPLIT
def train_test_split_dataloaders(dataset, frac):
    train = torch.utils.data.Subset(dataset, range(0, int(frac*len(dataset))))
    test = torch.utils.data.Subset(dataset, range(int(frac*len(dataset)), len(dataset)))

    trainloader = DataLoader(train, batch_size=batch_size, shuffle=True, drop_last=True, pin_memory=True)
    testloader = DataLoader(test, batch_size=batch_size, shuffle=True, drop_last=True, pin_memory=True)

    return trainloader, testloader

data = np.array(data)-1 #make sure you're working with the doppler shift values by subtracting 1
data = data/np.amax(data)
dataset = DatasetClass(data, targets)
trainloader, testloader = train_test_split_dataloaders(dataset,0.8)

In [4]:
#DEFINE A NEURAL NETWORK
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(50,10)
        self.fc2 = nn.Linear(10,6)
        # self.fc3 = nn.Linear(10,1)

    def forward(self,x):
        x = x.view(batch_size, -1)
        x = F.leaky_relu(self.fc1(x))
        x = F.leaky_relu(self.fc2(x))
        return x

#A DIFFERENT ITERATION OF THE NETWORK
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.cl1 = nn.Conv1d(1,3,3,1)

        self.cl2 = nn.Conv1d(3,5,3,1)
        self.fc1 = nn.Linear(20,6)
        
    def forward(self,x):
        x = x.view(batch_size, 1, -1)
        # print('batch input into the model', x.size())
        x = torch.relu(self.cl1(x))
        x = torch.max_pool1d(x,3)
        x = torch.relu(self.cl2(x))
        # print('after first conv1d', x.size())
        x = torch.max_pool1d(x,3)
        # print('after second conv1d', x.size())
        x = x.view(batch_size, -1)
        # print('size after flattening the conv layers', x.size())
        x = F=torch.relu(self.fc1(x))
        return x


#INSTANTIATE THE MODEL
# model = NeuralNetwork()
model = ConvNet()
print('model name: ', model)
device = get_device()
print('device being used: ', device)
model.to(device)

#OPTIMIZER AND LOSS FUNCTION
optimizer = torch.optim.Adam(model.parameters(), lr = lr)
loss_fn = nn.MSELoss()
accuracy, train_losses, test_losses = [],[],[]

model name:  ConvNet(
  (cl1): Conv1d(1, 3, kernel_size=(3,), stride=(1,))
  (cl2): Conv1d(3, 5, kernel_size=(3,), stride=(1,))
  (fc1): Linear(in_features=20, out_features=6, bias=True)
)
device being used:  cuda:0


In [5]:
#TRAINING LOOP
for i in range(epochs):
    for j, (x_train, y_train) in enumerate(trainloader):
        optimizer.zero_grad()
        # print('xtrain', x_train[:2])
        output = model(x_train.to(device))

        #calculate loss
        loss = loss_fn(output, y_train.to(device))

        #backpropagation
        loss.backward()
        optimizer.step()

    if i%10==0:
        train_losses.append(loss)
        # accuracy.append(acc)
        print(f'epoch:{i}, loss:{loss}')        

print('Finished!')

epoch:0, loss:6.448206901550293


KeyboardInterrupt: 

In [70]:

for j, (x_test, y_test) in enumerate(testloader):
    output = model(x_test.to(device))

    print(output, y_test)
    break


tensor([[0.1604, 4.1884, 1.1703, 3.8452, 0.2686, 0.0000],
        [0.1436, 1.2434, 0.6083, 1.3712, 0.2305, 0.0000],
        [0.1540, 2.1167, 0.7001, 2.0480, 0.2501, 0.0000],
        ...,
        [0.1475, 1.0961, 0.5478, 1.1534, 0.2237, 0.0000],
        [0.2030, 3.3791, 1.2854, 3.6143, 0.2873, 0.0000],
        [0.1639, 2.6989, 1.0693, 2.9999, 0.2795, 0.0000]], device='cuda:0',
       grad_fn=<ReluBackward0>) tensor([[0.0937, 0.9230, 1.8073, 3.5715, 0.2830, 3.2115],
        [0.2551, 2.0888, 0.5361, 3.3318, 0.2010, 2.6620],
        [0.0639, 4.2279, 0.4363, 4.0942, 0.2873, 1.3405],
        ...,
        [0.2395, 0.0476, 0.4477, 5.6376, 0.2339, 6.0969],
        [0.1356, 4.4905, 1.4027, 3.4462, 0.2998, 3.9642],
        [0.1270, 4.9481, 1.1211, 5.6671, 0.2651, 5.3434]])
